In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\joaog\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#read the file with the information from the game fifa_20
fifa20 = "data/players_20.csv"
fifa20_df = pd.read_csv(fifa20, encoding='utf8')

#sorting the df for the 200 most value players
fifa20_df_value = fifa20_df.sort_values(by='value_eur', ascending=False).head(200)

In [3]:
# cleaning the to select only columns of the data we need
fifa20_clean_df = fifa20_df_value[["short_name", "long_name","age", "club", "overall",\
                                   "potential", "value_eur", "nationality"]]
fifa20_clean_df

,short_name,long_name,age,club,overall,potential,value_eur,nationality
2,Neymar Jr,Neymar da Silva Santos Junior,27,Paris Saint-Germain,92,92,105500000,Brazil
0,L. Messi,Lionel Andrés Messi Cuccittini,32,FC Barcelona,94,94,95500000,Argentina
10,K. Mbappé,Kylian Mbappé,20,Paris Saint-Germain,89,95,93500000,France
5,K. De Bruyne,Kevin De Bruyne,28,Manchester City,91,91,90000000,Belgium
4,E. Hazard,Eden Hazard,28,Real Madrid,91,91,90000000,Belgium
...,...,...,...,...,...,...,...,...
276,K. Volland,Kevin Volland,26,Bayer 04 Leverkusen,82,83,26500000,Germany
270,M. Götze,Mario Götze,27,Borussia Dortmund,82,83,26000000,Germany
283,F. Kostić,Filip Kostić,26,Eintracht Frankfurt,82,83,26000000,Serbia
331,T. Ndombele,Tanguy NDombèlé Alvaro,22,Tottenham Hotspur,81,89,26000000,France


In [4]:
#read the file with the information scraped from transfermakt site
trsf = "data/transfermarkt.csv"
transfmarkt_df = pd.read_csv(trsf, encoding='utf8')
transfmarkt_df

,Name,Image,Position,Market Value
0,Kylian Mbappé,https://tmssl.akamaized.net//images/portrait/s...,Centre-Forward,$228.00m
1,Raheem Sterling,https://tmssl.akamaized.net//images/portrait/s...,Left Winger,$182.40m
2,Neymar,https://tmssl.akamaized.net//images/portrait/s...,Left Winger,$182.40m
3,Sadio Mané,https://tmssl.akamaized.net//images/portrait/s...,Left Winger,$171.00m
4,Mohamed Salah,https://tmssl.akamaized.net//images/portrait/s...,Right Winger,$171.00m
5,Harry Kane,https://tmssl.akamaized.net//images/portrait/s...,Centre-Forward,$171.00m
6,Kevin De Bruyne,https://tmssl.akamaized.net//images/portrait/s...,Attacking Midfield,$171.00m
7,Lionel Messi,https://tmssl.akamaized.net//images/portrait/s...,Right Winger,$159.60m
8,Jadon Sancho,https://tmssl.akamaized.net//images/portrait/s...,Right Winger,$136.80m
9,Antoine Griezmann,https://tmssl.akamaized.net//images/portrait/s...,Centre-Forward,$136.80m


In [5]:
# using fuzzywuzzy
# create a function to macht the names on the transfermarkt df with the long and short names on the fifa_20 df

def match_names(name, list_names1,list_names2, min_score=0):
    # 0 score incase we don't get any matches
    max_score1 = 0
    max_score2 = 0

    # Returning empty name for no match as well
    max_name1 = ""
    max_name2 = ""

    # Iternating over all names (transfermartk df) in two list of names (shor and long in fifa_20 df)
    for name1 in list_names1:
        #Finding fuzzy match score
        score1 = fuzz.ratio(name, name1)
        # Checking if we are above our threshold and have a better score
        if (score1 > min_score) & (score1 > max_score1):
            max_name1 = name1
            max_score1 = score1
            
    for name2 in list_names2:
        #Finding fuzzy match score
        score2 = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score2 > min_score) & (score2 > max_score2):
            max_name2 = name2
            max_score2 = score2
            
    # return the names matched for the two lists
    return (max_name1, max_score1, max_name2, max_score2)

In [6]:
# matche the names on diferent dataframes
dict_list = []

# iterating over players in transfermarkt
for name in transfmarkt_df.Name:
    # Use our method to find best match, we can set a threshold here 
    match = match_names(name, fifa20_clean_df.short_name,fifa20_clean_df.long_name, 50)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"Name" : name})
    dict_.update({"short_name" : match[0]})
    dict_.update({"score_short" : match[1]})
    dict_.update({"long_name" : match[2]})
    dict_.update({"score_long" : match[3]})
    dict_list.append(dict_)
    
merge_table_names = pd.DataFrame(dict_list)

# Display results
merge_table_names

,Name,short_name,score_short,long_name,score_long
0,Kylian Mbappé,K. Mbappé,73,Kylian Mbappé,100
1,Raheem Sterling,R. Sterling,77,Raheem Sterling,100
2,Neymar,Neymar Jr,80,,0
3,Sadio Mané,S. Mané,71,Sadio Mané,100
4,Mohamed Salah,M. Salah,67,Mohamed Salah Ghaly,79
5,Harry Kane,H. Kane,71,Harry Kane,100
6,Kevin De Bruyne,K. De Bruyne,81,Kevin De Bruyne,100
7,Lionel Messi,L. Messi,70,Lionel Andrés Messi Cuccittini,57
8,Jadon Sancho,J. Sancho,76,Jadon Sancho,100
9,Antoine Griezmann,A. Griezmann,76,Antoine Griezmann,100


In [7]:
# merge the matching names with the fifa_20 df and Trasnfermarkt df
fifa_names_df =  pd.merge(merge_table_names, fifa20_clean_df,  how='left',\
                             left_on=['short_name' or'long_name'], right_on = ['short_name'or'long_name'])

fifa_transfermark_df = pd.merge(fifa_names_df, transfmarkt_df,  how='left',\
                             left_on=['Name'], right_on = ['Name'])
fifa_transfermark_df

,Name,short_name,score_short,long_name_x,score_long,long_name_y,age,club,overall,potential,value_eur,nationality,Image,Position,Market Value
0,Kylian Mbappé,K. Mbappé,73,Kylian Mbappé,100,Kylian Mbappé,20,Paris Saint-Germain,89,95,93500000,France,https://tmssl.akamaized.net//images/portrait/s...,Centre-Forward,$228.00m
1,Raheem Sterling,R. Sterling,77,Raheem Sterling,100,Raheem Sterling,24,Manchester City,88,90,73000000,England,https://tmssl.akamaized.net//images/portrait/s...,Left Winger,$182.40m
2,Neymar,Neymar Jr,80,,0,Neymar da Silva Santos Junior,27,Paris Saint-Germain,92,92,105500000,Brazil,https://tmssl.akamaized.net//images/portrait/s...,Left Winger,$182.40m
3,Sadio Mané,S. Mané,71,Sadio Mané,100,Sadio Mané,27,Liverpool,88,88,62000000,Senegal,https://tmssl.akamaized.net//images/portrait/s...,Left Winger,$171.00m
4,Mohamed Salah,M. Salah,67,Mohamed Salah Ghaly,79,Mohamed Salah Ghaly,27,Liverpool,90,90,80500000,Egypt,https://tmssl.akamaized.net//images/portrait/s...,Right Winger,$171.00m
5,Harry Kane,H. Kane,71,Harry Kane,100,Harry Kane,25,Tottenham Hotspur,89,91,83000000,England,https://tmssl.akamaized.net//images/portrait/s...,Centre-Forward,$171.00m
6,Kevin De Bruyne,K. De Bruyne,81,Kevin De Bruyne,100,Kevin De Bruyne,28,Manchester City,91,91,90000000,Belgium,https://tmssl.akamaized.net//images/portrait/s...,Attacking Midfield,$171.00m
7,Lionel Messi,L. Messi,70,Lionel Andrés Messi Cuccittini,57,Lionel Andrés Messi Cuccittini,32,FC Barcelona,94,94,95500000,Argentina,https://tmssl.akamaized.net//images/portrait/s...,Right Winger,$159.60m
8,Jadon Sancho,J. Sancho,76,Jadon Sancho,100,Jadon Sancho,19,Borussia Dortmund,84,92,44500000,England,https://tmssl.akamaized.net//images/portrait/s...,Right Winger,$136.80m
9,Antoine Griezmann,A. Griezmann,76,Antoine Griezmann,100,Antoine Griezmann,28,FC Barcelona,89,89,69000000,France,https://tmssl.akamaized.net//images/portrait/s...,Centre-Forward,$136.80m


In [8]:
# clean the fifa_transfermarkt DF 
fifa_transfermark_df = fifa_transfermark_df [['Name', 'long_name_y','age', 'club', 'overall', 'potential', 'value_eur',\
                                              'nationality','Image', 'Position', 'Market Value']]
fifa_transfermark_df

,Name,long_name_y,age,club,overall,potential,value_eur,nationality,Image,Position,Market Value
0,Kylian Mbappé,Kylian Mbappé,20,Paris Saint-Germain,89,95,93500000,France,https://tmssl.akamaized.net//images/portrait/s...,Centre-Forward,$228.00m
1,Raheem Sterling,Raheem Sterling,24,Manchester City,88,90,73000000,England,https://tmssl.akamaized.net//images/portrait/s...,Left Winger,$182.40m
2,Neymar,Neymar da Silva Santos Junior,27,Paris Saint-Germain,92,92,105500000,Brazil,https://tmssl.akamaized.net//images/portrait/s...,Left Winger,$182.40m
3,Sadio Mané,Sadio Mané,27,Liverpool,88,88,62000000,Senegal,https://tmssl.akamaized.net//images/portrait/s...,Left Winger,$171.00m
4,Mohamed Salah,Mohamed Salah Ghaly,27,Liverpool,90,90,80500000,Egypt,https://tmssl.akamaized.net//images/portrait/s...,Right Winger,$171.00m
5,Harry Kane,Harry Kane,25,Tottenham Hotspur,89,91,83000000,England,https://tmssl.akamaized.net//images/portrait/s...,Centre-Forward,$171.00m
6,Kevin De Bruyne,Kevin De Bruyne,28,Manchester City,91,91,90000000,Belgium,https://tmssl.akamaized.net//images/portrait/s...,Attacking Midfield,$171.00m
7,Lionel Messi,Lionel Andrés Messi Cuccittini,32,FC Barcelona,94,94,95500000,Argentina,https://tmssl.akamaized.net//images/portrait/s...,Right Winger,$159.60m
8,Jadon Sancho,Jadon Sancho,19,Borussia Dortmund,84,92,44500000,England,https://tmssl.akamaized.net//images/portrait/s...,Right Winger,$136.80m
9,Antoine Griezmann,Antoine Griezmann,28,FC Barcelona,89,89,69000000,France,https://tmssl.akamaized.net//images/portrait/s...,Centre-Forward,$136.80m


In [9]:
# clean the Market Value column
real_prices = fifa_transfermark_df['Market Value'].values.T.tolist()
real_value = []
for price in real_prices:
    new = price.split("$")
    new1 = new[1].split("m")
    real_value.append(new1[0])
print(real_value)

['228.00', '182.40', '182.40', '171.00', '171.00', '171.00', '171.00', '159.60', '136.80', '136.80', '136.80', '125.40', '114.00', '114.00', '114.00', '114.00', '114.00', '114.00', '114.00', '102.60', '102.60', '102.60', '102.60', '102.60', '102.60']


In [10]:
# replace the old Market Value column from string to float
real_df = pd.DataFrame(real_value)
real_df = real_df[0].astype(float)
real_df = real_df*1000000


fifa_transfermark_df['Market Value'] = real_df
fifa_transfermark_df

C:\Users\joaog\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Name,long_name_y,age,club,overall,potential,value_eur,nationality,Image,Position,Market Value
0,Kylian Mbappé,Kylian Mbappé,20,Paris Saint-Germain,89,95,93500000,France,https://tmssl.akamaized.net//images/portrait/s...,Centre-Forward,228000000.0
1,Raheem Sterling,Raheem Sterling,24,Manchester City,88,90,73000000,England,https://tmssl.akamaized.net//images/portrait/s...,Left Winger,182400000.0
2,Neymar,Neymar da Silva Santos Junior,27,Paris Saint-Germain,92,92,105500000,Brazil,https://tmssl.akamaized.net//images/portrait/s...,Left Winger,182400000.0
3,Sadio Mané,Sadio Mané,27,Liverpool,88,88,62000000,Senegal,https://tmssl.akamaized.net//images/portrait/s...,Left Winger,171000000.0
4,Mohamed Salah,Mohamed Salah Ghaly,27,Liverpool,90,90,80500000,Egypt,https://tmssl.akamaized.net//images/portrait/s...,Right Winger,171000000.0
5,Harry Kane,Harry Kane,25,Tottenham Hotspur,89,91,83000000,England,https://tmssl.akamaized.net//images/portrait/s...,Centre-Forward,171000000.0
6,Kevin De Bruyne,Kevin De Bruyne,28,Manchester City,91,91,90000000,Belgium,https://tmssl.akamaized.net//images/portrait/s...,Attacking Midfield,171000000.0
7,Lionel Messi,Lionel Andrés Messi Cuccittini,32,FC Barcelona,94,94,95500000,Argentina,https://tmssl.akamaized.net//images/portrait/s...,Right Winger,159600000.0
8,Jadon Sancho,Jadon Sancho,19,Borussia Dortmund,84,92,44500000,England,https://tmssl.akamaized.net//images/portrait/s...,Right Winger,136800000.0
9,Antoine Griezmann,Antoine Griezmann,28,FC Barcelona,89,89,69000000,France,https://tmssl.akamaized.net//images/portrait/s...,Centre-Forward,136800000.0


In [11]:
# writing the new data to a new csv file
fifa_transfermark_df.to_csv('data/fifa_transfermark_df.csv',index=False)